In [1]:
import os
import string
import re   
import pickle
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
# nltk.download('punkt')

def save_binary(data, file, folder):
    location  = os.path.join(folder, (file+'.pickle') )
    with open(location, 'wb') as ff:
        pickle.dump(data, ff, protocol=pickle.HIGHEST_PROTOCOL)

def load_binary(file, folder):
    location  = os.path.join(folder, (file+'.pickle') )
    with open(location, 'rb') as ff:
        data = pickle.load(ff)
    return data

harakat= load_binary('diacritics','./')

def clear_tashkel(text):
    text = "".join(c for c in text if c not in harakat)
    return text

def get_tashkel(sentence):
    # harakat= load_binary('diacritics','./')
    output = []
    current_haraka = ""
    chIndex = 0
    mode = 0  # mode 0 is character meant (expecting to get character) and mode 1 is tashkeel ment (expecting to get tashkeel)
    while chIndex < (len(sentence)):
        characterTashkeels = []
        if mode == 1:
            while chIndex < (len(sentence)) and sentence[chIndex] in harakat:
                characterTashkeels.append(sentence[chIndex])
                chIndex += 1

            if (len(characterTashkeels) != 0):
                output.append(characterTashkeels)
            else:
                output.append("_") # no tashkeel for now
            # chIndex += 1
            mode = 0
        else:
            mode = 1
            chIndex += 1
    
    if mode == 1: # now I am exepcting tashkeel but the word ended before I find one
        output.append("_")  # _ symbolizes no tashkeel
    return output


arabic_alphabet_set=load_binary('arabic_letters','./')
arabic_alphabet = dict(zip(arabic_alphabet_set, list(range(len(arabic_alphabet_set)))))

def get_char_vector(char):
    if char in arabic_alphabet:
        vector = [0 for _ in range(len(arabic_alphabet))]
        vector[arabic_alphabet[char] ] = 1
        return vector
    else:
        return list(np.ones(len(arabic_alphabet),dtype=int) )


In [2]:
import nltk
import string
import re
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
# read the training set
def readFile(file_path):
    # Open the file for reading
    file_content = None
    with open(file_path, 'r', encoding='utf-8') as file:
        # Read the content of the file
        file_content = file.read()
    return (file_content)

def cleanText(data):
    punctuations=["،","؛","؟",".",",","!",":"," "]
    listTobeChecked = set(tuple(list(harakat)+list(arabic_alphabet.keys())+punctuations))
    for char in data:
        #check if char is not in arabic alphabet or harakat
        if char not in (listTobeChecked):
            data = data.replace(char,'')
    # split text into sentences using regex
    
    data = re.sub('  +',' ', data)
    
    data = re.split(r'[،؛؟.,!:]+', data)
    
    return data
def get_sentences(sentences):
    words_per_sentence = []
    tashkeel_per_sentence = []
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence == '': continue # new line may be followed by full stop
        sentence = word_tokenize(sentence)
        words_per_sentence.append(sentence)
    
    tashkeel_per_sentence = [[None for _ in w ]for w in words_per_sentence]
    for i, x in enumerate(words_per_sentence):
        for j, y in enumerate(x):
            tashkeel = get_tashkel(y)
            words_per_sentence[i][j] = clear_tashkel(y)
            tashkeel_per_sentence[i][j] = tashkeel
    return words_per_sentence, tashkeel_per_sentence

# sent=["عَُمرً"] 

# %%
from gensim.models import  FastText
# Define and train Word2Vec model
def makeModel(dataset):
    model = FastText(sentences=dataset, vector_size=100, window=15, min_count=1, workers=40)
    # Save the trained model (optional)
    model.save("model.model")
    return model

def getCharacterEncoding(word):
    word_embedding = list()
    for w in word:
        word_embedding.append(get_char_vector(w))
    return word_embedding


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, SimpleRNNCell, RNN, Dense,Layer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
max_sentence_length = 100
embedding_dim = 100
max_char_length = 15
char_vocab_size = 36
num_diacritics = 15
#very small number
lstm_units = 32

In [5]:
sent=load_binary('sentences_without_diacritics','cleaned_data')

In [6]:
model_embedding =  FastText.load("model.model")

In [7]:
from gensim.models import Word2Vec, FastText

In [8]:
import csv

def writeToCSV(data):
    with open('submission.csv', mode='w', newline='') as csv_file:  # Added newline=''
        fieldnames = ['ID', 'label']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(len(data)):
            writer.writerow({'ID': i, 'label': data[i]})

In [20]:
from utils import *
from preprocessing import *
from gensim.models import Word2Vec, FastText
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

max_sentence_length = 30
embedding_dim = 100
max_char_length = 15
char_vocab_size = 36
num_diacritics = 15
model1h5=load_model('model11pm.h5')


def split_lists(input_list, max_length):
    result_lists = []
    for inner_list in input_list:
        for i in range(0, len(inner_list), max_length):
            result_lists.append(inner_list[i:i + max_length])
    return result_lists


def test_sent(path):
    testset = readFile(path)
    sent=cleanText(testset)
    
    sent, _ = get_sentences(sent)
    sent=split_lists(sent,max_sentence_length)

    embeddding_model=FastText.load('model.model')

    def getEmbeddings(sentences, word2vecmodel):
        embeddingSentences = [] # list of all sentences
        keys = word2vecmodel.wv.key_to_index
        for s in sentences:
            embeddingTemp = []  # list for one sentence
            for w in s:
                embeddingTemp.append(word2vecmodel.wv[w])
            embeddingSentences.append(embeddingTemp)
        return embeddingSentences

    embeddingsSentences = getEmbeddings(sent, embeddding_model)

    X_sentence = []  # List for sentence_input
    X_char = []      # List for char_input
    X_scalar = []    # List for scalar_input

    X_chars=[]
    for i in range(len(embeddingsSentences)):
        for j in range(len(embeddingsSentences[i])):
            for char in sent[i][j]:
                X_sentence.append(embeddingsSentences[i])
                X_char.append(get_char_vector(char))
                X_chars.append(getCharacterEncoding(sent[i][j]))
                X_scalar.append([j])
    X_character_padded = pad_sequences(X_chars, maxlen=max_char_length, padding='post', dtype='float16')
    X_chars2 = np.array(X_character_padded)
    X_sentence_padded = pad_sequences(X_sentence, maxlen=max_sentence_length, padding='post', dtype='float16')
    # Convert lists to numpy arrays
    X_sentence = np.array(X_sentence_padded)
    X_char = np.array(X_char)
    X_scalar = np.array(X_scalar)

    return X_sentence, X_char, X_scalar,X_chars2
X_sentence, X_char, X_scalar,X_chars2=test_sent("Delivery/sample_testset_with_submission/test_no_diacritics.txt")
predictions=model1h5.predict([X_sentence,X_char,X_chars2,X_scalar])
pred=np.argmax(predictions, axis=-1)

writeToCSV(pred)

13043/13043 [==============================] - 200s 15ms/step
